In [4]:
import os

import openai
from dotenv import load_dotenv, find_dotenv

/Users/alhudam/Desktop/projects/llm-tutorial/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [6]:
def get_completion(prompt, model="gpt-3.5-turbo"):
   messages = [{'role': 'user', 'content': prompt}]

   response = openai.ChatCompletion.create(
      model=model,
      messages=messages,
      temperature=0
   )

   return response.choices[0].message['content']

In [7]:
print(get_completion(prompt='what is 1+1'))

1 + 1 equals 2.


In [ ]:
from langchain.chat_models import ChatOpenAI

In [79]:
chat = ChatOpenAI(temperature=0.0)


In [11]:
template_string = """Translate the text \
that is delimited by triple backticks \ 
into a style that is {style}. \
text: ```{text}```
"""

In [12]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [13]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks \\ \ninto a style that is {style}. text: ```{text}```\n', template_format='f-string', validate_template=True)

In [14]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [15]:
customer_style = """American English \
in a calm and respectful tone
"""

In [16]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need your help \ 
right now, matey!
"""

In [17]:
customer_messages = prompt_template.format_messages(style=customer_style, text=customer_email)

In [18]:
print(customer_messages)

[HumanMessage(content="Translate the text that is delimited by triple backticks \\ \ninto a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need your help \\ \nright now, matey!\n```\n", additional_kwargs={}, example=False)]


In [19]:
customer_response = chat(customer_messages)

In [20]:
print(customer_response)

content="I'm really frustrated that my blender lid flew off and made a mess of my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, my friend!" additional_kwargs={} example=False


In [21]:
service_reply = """Hey there customer, \ 
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that you misused the blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [22]:
service_style_pirate = """\
a polite tone \
that speaks in English pirate\
"""

In [24]:
service_messages = prompt_template.format_messages(style=service_style_pirate, text=service_reply)

service_messages

[HumanMessage(content="Translate the text that is delimited by triple backticks \\ \ninto a style that is a polite tone that speaks in English pirate. text: ```Hey there customer, \\ \nthe warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused the blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!\n```\n", additional_kwargs={}, example=False)]

In [26]:
service_response = chat(service_messages)
print(service_response)

content="Ahoy there, matey! I be sorry to inform ye, but the warranty be not coverin' the expenses o' cleanin' yer galley, as 'tis yer own fault fer misusin' the blender by forgettin' to put the lid on afore startin' it. Aye, tough luck! Fare thee well, me heartie!" additional_kwargs={} example=False


In [28]:
{
    "gift": False,
    "delivery_days": 5,
    "price_value": "pretty affordable"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable'}

In [38]:
customer_review = """\
This leaf blower is pretty amazing. It has four settings:\
candle blower, gentle breeze, windy city, and tornado.\
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

In [57]:
review_template = """
For the following text, extract the following information: \
gift: Was the item purchased as a gift/present for someone else? Answer true if its a yes or false if its a no \
delivery_days: How many days did it take for the product to deliver? Answer in number \
price_value: Extract any sentences about the value or price, keep it short \

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [58]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)

In [59]:
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='\nFor the following text, extract the following information: gift: Was the item purchased as a gift/present for someone else? Answer true if its a yes or false if its a no delivery_days: How many days did it take for the product to deliver? Answer in number price_value: Extract any sentences about the value or price, keep it short \nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [60]:
messages = prompt_template.format_messages(style=customer_style, text=customer_review)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": "It's slightly more expensive than the other leaf blowers out there"
}


In [70]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser, PydanticOutputParser

In [73]:
gift_schema = ResponseSchema(name="gift", description="Was the item purchased as a gift/present for someone else? Answer true if its a yes, False if not or unknown")
delivery_days_schema =  ResponseSchema(name="delivery_days", description="How many days did it take for the product to deliver? Answer in number")
price_value_schema =  ResponseSchema(name="price_value", description="Extract any sentences about the value or price, keep it short")

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

# output_parser = PydanticOutputParser(schema=response_schemas)

In [66]:
format_instructions = output_parser.get_format_instructions()

NameError: name 'output_parser' is not defined